In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Set seeds
np.random.seed(42)

# Define data directories
train_dir = '/content/drive/MyDrive/Input/nodulenet/train/'
val_dir = '/content/drive/MyDrive/Input/nodulenet/val/'
test_dir = '/content/drive/MyDrive/Input/nodulenet/test/'

# Define batch size
batch_size = 32

# Read labels
train_labels = pd.read_csv('/content/drive/MyDrive/Input/nodulenet/trainlabels.txt', sep=' ', header=None, names=['filename', 'label'])
val_labels = pd.read_csv('/content/drive/MyDrive/Input/nodulenet/vallabels.txt', sep=' ', header=None, names=['filename', 'label'])
test_labels = pd.read_csv('/content/drive/MyDrive/Input/nodulenet/testlabels.txt', sep=' ', header=None, names=['filename', 'label'])


In [2]:
# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


In [3]:
# Define data iterators
train_iterator = train_datagen.flow_from_dataframe(dataframe=train_labels,
                                                    directory=train_dir,
                                                    x_col='filename',
                                                    y_col='label',
                                                    target_size=(64, 64),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

val_iterator = val_datagen.flow_from_dataframe(dataframe=val_labels,
                                                directory=val_dir,
                                                x_col='filename',
                                                y_col='label',
                                                target_size=(64, 64),
                                                batch_size=batch_size,
                                                class_mode='categorical')

test_iterator = test_datagen.flow_from_dataframe(dataframe=test_labels,
                                                  directory=test_dir,
                                                  x_col='filename',
                                                  y_col='label',
                                                  target_size=(64, 64),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 5188 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1298 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1623 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [5]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

# Load pre-trained VGG16 model without the top layer
base_model = VGG16(include_top=False, input_shape=(64,64,3))

# Freeze the base model weights
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 15,239,746
Trainable params: 525,058
Non-trainable params: 14,714,688
_____________________________________

In [18]:
base_model_tl = VGG16(include_top=False, input_shape=(64,64,3))

In [20]:
for layer in base_model_tl.layers:
  layer.trainable = False

In [21]:
model_tl = Sequential()
model_tl.add(base_model_tl)
model_tl.add(Flatten())
model_tl.add(Dense(256, activation='relu'))
model_tl.add(Dropout(0.5))
model_tl.add(Dense(2, activation='softmax'))

In [22]:
model_tl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])